In [ ]:
import numpy as np
import pandas as pd

train_df = pd.read_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project_home_credit/data/train_bo2.pkl')
test_df = pd.read_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project_home_credit/data/test_bo2.pkl')
print(train_df.shape, test_df.shape)
train_df.head(2)

In [ ]:
# null_value_stats = train_df.isnull().sum(axis=0)

# null_value_stats

# null_value_stats[null_value_stats != 0]

# train_df.fillna(-999, inplace=True)
# test_df.fillna(-999, inplace=True)

In [ ]:
train_df = train_df.sample(frac=0.05)

In [ ]:
X = train_df.drop('TARGET', axis=1)
y = train_df['TARGET']

In [ ]:
# import gc
# del train_df; gc.collect()

In [ ]:
X.shape, y.shape

In [ ]:
cat_cols = ["FLAG_OWN_CAR","FLAG_OWN_REALTY","NAME_TYPE_SUITE","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","OCCUPATION_TYPE","FLAG_DOCUMENT_2","FLAG_DOCUMENT_3","FLAG_DOCUMENT_4","FLAG_DOCUMENT_5","FLAG_DOCUMENT_6","FLAG_DOCUMENT_7","FLAG_DOCUMENT_8","FLAG_DOCUMENT_9","FLAG_DOCUMENT_10","FLAG_DOCUMENT_11","FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13","FLAG_DOCUMENT_14","FLAG_DOCUMENT_15","FLAG_DOCUMENT_16","FLAG_DOCUMENT_17","FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19","FLAG_DOCUMENT_20","FLAG_DOCUMENT_21", "CAT_MONTHS_BIRTH", "CNT_CHILDREN", "CODE_GENDER", "CONTACT", "EMERGENCYSTATE_MODE", "FLAG_EMAIL", "FLAG_OTHER", "FONDKAPREMONT_MODE", "HOUR_APPR_PROCESS_START", "HOUSETYPE_MODE", "LIVE_CITY_NOT_WORK_CITY", "LIVE_REGION_NOT_WORK_REGION", "NAME_CONTRACT_TYPE", "ORGANIZATION_TYPE", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "REGION_RATING_CLIENT_W_CITY_times_livingsum_AVG", "REGION_RATING_CLIENT_W_CITY_times_livingsum_MEDI", "REGION_RATING_CLIENT_W_CITY_times_livingsum_MODE", "REGION_RATING_CLIENT_times_livingsum_AVG", "REGION_RATING_CLIENT_times_livingsum_MEDI", "REGION_RATING_CLIENT_times_livingsum_MODE", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "WALLSMATERIAL_MODE", "WEEKDAY_APPR_PROCESS_START", "count_CODE_GENDER_NAME_EDUCATION_TYPE", "count_CODE_GENDER_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE", "count_CODE_GENDER_NAME_INCOME_TYPE", "count_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE", "unique_CODE_GENDER_NAME_EDUCATION_TYPE", "unique_CODE_GENDER_NAME_INCOME_TYPE", "unique_CODE_GENDER_x", "unique_CODE_GENDER_y", "unique_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE", "unique_NAME_EDUCATION_TYPE_x", "unique_NAME_EDUCATION_TYPE_y", "unique_NAME_INCOME_TYPE_x", "unique_NAME_INCOME_TYPE_y"]

In [ ]:
categorical_features_indices = [X.columns.tolist().index(col) for col in cat_cols]

print(categorical_features_indices)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

X_test = test_df

In [ ]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import roc_auc_score

In [ ]:
params = {
    'iterations': 50,
    'learning_rate': 0.1,
#     'custom_metric': 'AUC',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'logging_level': 'Verbose',
    'thread_count': 15,
    'use_best_model': True,
    'save_snapshot': False 
}

model = CatBoostClassifier(**params).fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_validation, y_validation))

In [ ]:
params = {
    'iterations': 3000,
#     'custom_metric': 'AUC',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'logging_level': 'Verbose',
    'thread_count': 15,
    'use_best_model': True,
    'save_snapshot': True
}

model = CatBoostClassifier(**params).fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_validation, y_validation))

In [ ]:
params = {
    'iterations': 10000,
#     'custom_metric': 'AUC',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'logging_level': 'Verbose',
    'thread_count': 15,
    'use_best_model': True,
    'save_snapshot': True
}

model = CatBoostClassifier(**params).fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_validation, y_validation))

In [ ]:
m1 = CatBoostClassifier()

In [ ]:
m1.fit()

In [ ]:
model.tree_count_

In [ ]:
len(model.get_test_eval())

In [ ]:
pred = model.predict_proba(test_df)[:,1]

In [ ]:
sub = pd.read_csv('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project_home_credit/data/sample_submission.csv')

In [ ]:
sub.to_csv('iter9103_catboost_val7961.csv.gz', index=False, compression='gzip')

In [ ]:
# fe = model.feature_importances_
# fe_dict = dict(zip(X_train.columns.tolist(), fe))
# sorted_fe_dict = sorted(fe_dict.items(), key=lambda x:x[1])#, reverse=True)
# sorted_fe_dict[300:350]
# zero_importance_feats_not_cat = []
# for i,(fe,score) in enumerate(sorted_fe_dict):
#     if score == 0 and fe not in cat_cols:
#         print(i, fe)
#         zero_importance_feats_not_cat.append(fe)
# len(zero_importance_feats_not_cat)

# import pickle
# pickle.dump(sorted_fe_dict, open('sorted_catb_fe_dict.pkl', mode='wb'))
# templ = pickle.load(open('sorted_catb_fe_dict.pkl', mode='rb'))
# train_df_fe0removed = train_df.drop(zero_importance_feats_not_cat, axis=1)
# test_df_fe0removed = test_df.drop(zero_importance_feats_not_cat, axis=1)
# print(train_df_fe0removed.shape, test_df_fe0removed.shape)
# train_df_fe0removed.to_pickle('train_bo2_fe0removed.pkl')
# test_df_fe0removed.to_pickle('test_bo2_fe0removed.pkl')

In [ ]:
params = {
    'iterations': 30,
    'depth': np.random.randint(6,11),
    'l2_leaf_reg': np.random.randint(0, 31)/10,
#     'custom_metric': 'AUC',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'logging_level': 'Verbose',
    'thread_count': 15
}

model = CatBoostClassifier(**params)

cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    model.get_params(),
    plot=True
)

In [ ]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-AUC-mean']),
    cv_data['test-AUC-std'][cv_data['test-AUC-mean'].values.argmax()],
    cv_data['test-AUC-mean'].values.argmax()
))

In [ ]:
model.get_params()

In [ ]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

In [ ]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

In [ ]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

In [ ]:
best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

In [ ]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

In [ ]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

In [ ]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

In [ ]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# Get baseline (only with prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type='RawFormulaVal')
# Fit new model
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline);

In [ ]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric='Accuracy',
        random_seed=42,
        logging_level='Silent'
    )
    
    cv_data = cv(
        Pool(X, y, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

In [ ]:
trials = hyperopt.Trials()

In [ ]:
best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials
)

print(best)

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='Accuracy',
    random_seed=42,
    logging_level='Silent'
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))